# Lawyer Entity Linking

#### Setup

In [2]:
#import pdb; pdb.set_trace()

%matplotlib inline  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import json
import psycopg2
from sqlalchemy.engine.url import URL

# connection to the database
# connection string for use in pandas:
con = str(URL(drivername='postgresql', 
              username=os.environ['DB_QIR_USERNAME'], 
              password=os.environ['DB_QIR_PASSWORD'], 
              host='www.quantleap.nl', 
              #host='localhost', 
              database='qir'))

# cursor for use with psycopg2
conn = psycopg2.connect(con)
cur = conn.cursor()  
print('CONNECTION ESTABLISHED')

CONNECTION ESTABLISHED


In [27]:
pd.set_option("display.max_rows", 500)
pd.get_option("display.max_rows")

500

In [3]:
df = pd.read_sql("""select aa.id, source, cir_initials, cir_full_family_name, bar_code, full_name, 
                    has_same_normalized_name, is_verified from administrator_associations aa 
                    join nova_lawyers n on aa.nova_lawyer_id = n.id
                    order by source, cir_full_family_name, cir_initials""", con)


In [4]:
df.head()

,id,source,cir_initials,cir_full_family_name,bar_code,full_name,has_same_normalized_name,is_verified
0,2113,advocatenzoeken,r.j.g.,abeln,A20770,Mr. R.J.G. Abeln,True,True
1,2114,advocatenzoeken,h.p.,abma,A26396,Mr. H.P. Abma,True,True
2,2120,advocatenzoeken,s.w.h.,arends,A20574,Mr. S.W.H. Arends,True,True
3,2131,advocatenzoeken,n.,bakker,A13296,Mr. Niels Bakker,False,False
4,2682,advocatenzoeken,r.a.,baltes,A16624,de heer mr. R.A. Baltes,True,True


In [5]:
print(len(df))
df.groupby(['source']).count()['id']

1563


source
advocatenzoeken     280
manual              149
nova               1134
Name: id, dtype: int64

In [6]:
df.columns

Index(['id', 'source', 'cir_initials', 'cir_full_family_name', 'bar_code',
       'full_name', 'has_same_normalized_name', 'is_verified'],
      dtype='object')

In [7]:
pt = pd.pivot_table(df, values='id', index=['source'],
...                     columns=['has_same_normalized_name'], aggfunc=len)

In [8]:
pt

has_same_normalized_name,False,True
source,,
advocatenzoeken,31.0,249.0
manual,149.0,NaN
nova,76.0,1058.0


In [9]:
pt.columns

Index([False, True], dtype='object', name='has_same_normalized_name')

In [10]:
pt.index

Index(['advocatenzoeken', 'manual', 'nova'], dtype='object', name='source')

## which administrators have been taken of their case in current active cases

In [13]:
# which administrators in current active cases stopped working on the case

df = pd.read_sql("""select nl.id as nova_lawyer_id, nl.full_name, firm.full_name, count(*) from active_company_bankrupt_insolvents_view i
  join administrators_receivers ar on ar.insolvent_id = i.id
  join nova_lawyers nl on ar.nova_lawyer_id = nl.id
  join nova_law_firms firm on nl.law_firm_id = firm.id
  where ar.date_end is not null
    and nl.full_name != 'ONBEKEND'
group by 1, 2, 3
order by 4 desc
limit 20;""", con)
df  # print(df.to_csv())

,nova_lawyer_id,full_name,full_name,count
0,989,de heer mr. L.C. de Jong,Roest Crollius De Jong Advocaten,111
1,1579,Mr. S.H.F. Hoppenbrouwers,Holla Advocaten,52
2,1265,mevrouw mr. K.M.J.R. Maessen,De Voort Advocaten|Mediators,48
3,1522,Mr. Louis Deterink,Deterink Advocaten en Notarissen N.V.,45
4,1213,de heer mr. B.W.G.P. Meijs,Thuis Partners Advocaten,32
5,646,de heer mr. J.A.D.M. Daniels,Daniels Huisman Advocaten,32
6,1259,de heer mr. E. Douma,Alderse Baas Advocaten,29
7,448,mevrouw mr. A.M.T. Weersink,Hekkelman Advocaten N.V.,28
8,1022,de heer mr. P.C.H.H. Kager,KVH Beheer BV,23
9,1667,Mr. E.J. Rotshuizen,Rotshuizen Geense Advocaten,20


In [19]:
# lawyer details
sql = """select json_data from nova_lawyers where id = {id}""".format(id=989)
details = pd.read_sql(sql, con)['json_data'][0]
json.loads(details)


{'type': 'person',
 'full_name': 'de heer mr. L.C. de Jong',
 'full_address': 'Westdam 3-e 3441 GA WOERDEN - Nederland',
 'lat': '52.0880124000',
 'lng': '4.8823695000',
 'oath': '1991-10-16 00:00:00',
 'oath_first': '1991-10-16 00:00:00',
 'district': 'Midden-Nederland',
 'rechtsgebieden': '{"Huurrecht":["Huurrecht - algemeen","Huur van woonruimte","Verhuur van woonruimte","Huur van bedrijfsruimte","Verhuur van bedrijfsruimte"],"Insolventierecht\\/Beslag- en executierecht":["Faillissementsrecht – curator","Faillissementsrecht – schuldenaar","Faillissementsrecht - schuldeiser","Surséance van betaling","Beslag- en executierecht"],"Ondernemingsrecht":["Ondernemingsrecht - algemeen","Vennootschapsrecht","Rechtspersonenrecht","Fusies en overnames"],"Personen-, familie-, erfrecht":["Personenrecht - algemeen","Familierecht - algemeen","Echtscheidingen, alimentatiezaken, omgangsregelingen","Echtscheidingen met huwelijkse voorwaarden","Familiezaken met internationale aspecten","Boedelscheiding

In [29]:
# display cases of which lawyer is taken:
sql = """select case_number, start_date_insolvency, end_date_insolvency, ar.date_end from 
         insolvents i join administrators_receivers ar on ar.insolvent_id = i.id
         where nova_lawyer_id = {nova_lawyer_id}
             and ar.date_end is not null
             order by ar.date_end desc;""".format(nova_lawyer_id=989)
print(pd.read_sql(sql, con)[:10].to_csv())


,case_number,start_date_insolvency,end_date_insolvency,date_end
0,F.16/18/303,2018-07-03,,2018-07-03
1,F.16/18/302,2018-07-03,,2018-07-03
2,F.16/18/301,2018-07-03,,2018-07-03
3,F.13/16/356,2016-08-11,,2017-10-13
4,F.13/16/358,2016-08-11,,2017-10-13
5,F.13/16/357,2016-08-11,,2017-10-13
6,F.16/16/396,2016-07-12,,2017-10-10
7,F.16/12/562,2012-09-21,,2017-10-10
8,F.16/11/21,2011-01-13,,2017-10-10
9,F.16/13/406,2013-04-19,,2017-10-10



In [35]:
# check publications of case
sql = """select date, description from 
         publications p join insolvents i on p.insolvent_id = i.id
         where i.case_number = '{case_number}'
             order by date desc;""".format(case_number='F.13/09/353')
pd.read_sql(sql, con)



,date,description
0,2017-12-14,Opheffing faillissement wegens gebrek aan bate...
1,2017-10-04,vervanging curator mr. C. de Jong door


In [78]:
# which administrators in current active cases stopped working on the case - for cases after 1/1/2017

df = pd.read_sql("""select nl.full_name, firm.full_name, count(*) from active_company_bankrupt_insolvents_view i
  join administrators_receivers ar on ar.insolvent_id = i.id
  join nova_lawyers nl on ar.nova_lawyer_id = nl.id
  join nova_law_firms firm on nl.law_firm_id = firm.id
  where ar.date_end is not null
    and nl.full_name != 'ONBEKEND'
    and i.start_date_insolvency > '2017-01-01'
group by 1, 2
order by 3 desc
limit 50;""", con)
print(df.to_csv())


,full_name,full_name,count
0,mevrouw mr. K.M.J.R. Maessen,De Voort Advocaten|Mediators,11
1,mevrouw mr. A.S. van Hilten,DVDW Advocaten,10
2,de heer mr. E. Douma,Alderse Baas Advocaten,10
3,Mr. R.M. Hilbrink,DV advocaten B.V.,8
4,Mr. A.C.A. Klerks - Valks,Turnaround Advocaten,7
5,Mr. A.R. de Vries - Oosterveld,TeekensKarstens advocaten notarissen,7
6,Mr. Roel Jonker,Van Diepen Van der Kroef Advocaten,7
7,de heer mr. F.J.B. Buitenhuis,Delissen Martens advocaten belastingadviseurs mediation,6
8,mevrouw mr. M. Waijers,NS Groep N.V. / NS Legal,6
9,Mr. R.A. van Wijk,Lexington Advocaten,5
10,Mr. Y.C.M. Heruer,Okkerse & Schop Advocaten,5
11,de heer mr. L.C. de Jong,Roest Crollius De Jong Advocaten,5
12,de heer mr. P. Beerens,Ploum,5
13,Mr. S.F.M. ten Berge,DVDW Advocaten,5
14,Mr. W.L. Eikendal,Boels Zanders Advocaten,5
15,Mr. J.H. Steverink,Stellicher advocaten,5
16,Mr. T.H. Otten,Houthoff Buruma,5
17,Mr. M. Eefting,Loyens & Loeff N.V.,4
18,Mr. J.H. Hemmes,Hemmes & Flipse Advocaten,4
19,de hee